## UAP Generate & Evaluate (whitebox)

attacks
- chessboard, eps = 2-12
- random noise, eps = 2-12

dataset
- cifar10
- svhn

model_name
- resnet18
- [soft-pruning]
    - resnet18_sfP
    - resnet18_sfP-mixup
    - resnet18_sfP-cutout
- [post-training pruning]
    - resnet18_PX_0.Y
        - X = 2, 3, 4
        - Y = 3, 6, 9

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import json

from utils_attack import uap_batch
from utils_model import *

dir_results = './results/'
dir_uap = './uaps/'

In [2]:
nb_epoch = 10
y_target = -1

In [3]:
model_names =  ['resnet18', 'resnet18_sfP', 'resnet18_sfP-mixup', 'resnet18_sfP-cutout']
for layer in [2, 3, 4]:
    for prune_pct in ['0.3', '0.6', '0.9']:
        model_names.append('resnet18_P%i-%s' % (layer, prune_pct))
model_names

['resnet18',
 'resnet18_sfP',
 'resnet18_sfP-mixup',
 'resnet18_sfP-cutout',
 'resnet18_P2-0.3',
 'resnet18_P2-0.6',
 'resnet18_P2-0.9',
 'resnet18_P3-0.3',
 'resnet18_P3-0.6',
 'resnet18_P3-0.9',
 'resnet18_P4-0.3',
 'resnet18_P4-0.6',
 'resnet18_P4-0.9']

In [4]:
def eval_uap(uap):    
    top, top_probs, top1acc, top5acc, outputs, labels = evaluate(model, testloader, uap = uap)

    print('Top 1 accuracy', sum(top1acc) / len(labels))
    print('Top 5 accuracy', sum(top5acc) / len(labels))

    dict_entry = {}
    dict_entry['model_name'] = model_name
    dict_entry['dataset'] = dataset

    dict_entry['atk_name'] = atk_name
    dict_entry['atk_param'] = y_target
    dict_entry['eps'] = eps
    if atk_name[:3] == 'sgd': dict_entry['nb_epoch'] = nb_epoch

    dict_entry['top1acc'] = sum(top1acc) / len(labels)
    dict_entry['top5acc'] = sum(top5acc) / len(labels)
    dict_entry['UER'] = 1 - sum(top1acc) / len(labels)
    print('UER ', dict_entry['UER'])
    print('\n')
    if y_target >= 0: dict_entry['tgt_success'] = sum(outputs == y_target) / len(labels)

    # Output distribution
    for i in range(10):
        dict_entry['label_dist%i' % i] = sum(outputs == i) / len(labels)

    all_results.append(dict_entry)

In [5]:
all_results = []

for dataset in ['cifar10', 'svhn']:
    testloader = get_testdata(dataset)
        
    for model_name in model_names:
        model = get_model(model_name, dataset)

        # Save directory
        uap_save_path = dir_uap + dataset + '/' + model_name
        if not os.path.exists(uap_save_path):
            os.mkdir(uap_save_path)
            
        # 1. Chessboard attack
        atk_name = 'chess'
        uap = torch.load(dir_uap + 'chess.pth')
        for eps in np.arange(2, 13, 2):
            eval_uap(uap * eps / 255)

        # 2. Random noise
        atk_name = 'random'
        torch.manual_seed(0)
        uap = torch.sign(torch.rand_like(uap) - 0.5)
        for eps in np.arange(2, 13, 2):
            eval_uap(uap * eps / 255)
            
        pd.DataFrame(all_results).to_csv(dir_results + 'eval_noise.csv')

Files already downloaded and verified
Loading cifar10/resnet18.pth
 Completed: 0.00%
Top 1 accuracy 0.8446
Top 5 accuracy 0.9908
UER  0.15539999999999998


 Completed: 0.00%
Top 1 accuracy 0.6409
Top 5 accuracy 0.9305
UER  0.3591


 Completed: 0.00%
Top 1 accuracy 0.4126
Top 5 accuracy 0.7941
UER  0.5873999999999999


 Completed: 0.00%
Top 1 accuracy 0.2326
Top 5 accuracy 0.6466
UER  0.7674


 Completed: 0.00%
Top 1 accuracy 0.1431
Top 5 accuracy 0.5707
UER  0.8569


 Completed: 0.00%
Top 1 accuracy 0.1138
Top 5 accuracy 0.5361
UER  0.8862


 Completed: 0.00%
Top 1 accuracy 0.938
Top 5 accuracy 0.998
UER  0.062000000000000055


 Completed: 0.00%
Top 1 accuracy 0.92
Top 5 accuracy 0.9971
UER  0.07999999999999996


 Completed: 0.00%
Top 1 accuracy 0.8891
Top 5 accuracy 0.9938
UER  0.1109


 Completed: 0.00%
Top 1 accuracy 0.8429
Top 5 accuracy 0.9906
UER  0.15710000000000002


 Completed: 0.00%
Top 1 accuracy 0.7839
Top 5 accuracy 0.9817
UER  0.21609999999999996


 Completed: 0.00%
Top 1

Top 1 accuracy 0.1449
Top 5 accuracy 0.5546
UER  0.8551


Loading cifar10/resnet18_P3-0.3.pth
 Completed: 0.00%
Top 1 accuracy 0.8506
Top 5 accuracy 0.9914
UER  0.14939999999999998


 Completed: 0.00%
Top 1 accuracy 0.6599
Top 5 accuracy 0.9316
UER  0.34009999999999996


 Completed: 0.00%
Top 1 accuracy 0.4166
Top 5 accuracy 0.8071
UER  0.5833999999999999


 Completed: 0.00%
Top 1 accuracy 0.226
Top 5 accuracy 0.6572
UER  0.774


 Completed: 0.00%
Top 1 accuracy 0.1444
Top 5 accuracy 0.5788
UER  0.8556


 Completed: 0.00%
Top 1 accuracy 0.116
Top 5 accuracy 0.5439
UER  0.884


 Completed: 0.00%
Top 1 accuracy 0.9326
Top 5 accuracy 0.998
UER  0.06740000000000002


 Completed: 0.00%
Top 1 accuracy 0.915
Top 5 accuracy 0.9959
UER  0.08499999999999996


 Completed: 0.00%
Top 1 accuracy 0.8806
Top 5 accuracy 0.9933
UER  0.11939999999999995


 Completed: 0.00%
Top 1 accuracy 0.8336
Top 5 accuracy 0.9882
UER  0.1664


 Completed: 0.00%
Top 1 accuracy 0.7721
Top 5 accuracy 0.978
UER  0.2279




 Completed: 0.00%
Top 1 accuracy 0.9682314074984635
Top 5 accuracy 0.9963506453595574
UER  0.03176859250153652


 Completed: 0.00%
Top 1 accuracy 0.9682314074984635
Top 5 accuracy 0.9962738168408113
UER  0.03176859250153652


 Completed: 0.00%
Top 1 accuracy 0.968000921942225
Top 5 accuracy 0.9962738168408113
UER  0.03199907805777502


 Completed: 0.00%
Top 1 accuracy 0.9680777504609711
Top 5 accuracy 0.9962738168408113
UER  0.03192224953902889


 Completed: 0.00%
Top 1 accuracy 0.9681161647203442
Top 5 accuracy 0.9962738168408113
UER  0.03188383527965577


 Completed: 0.00%
Top 1 accuracy 0.9674631223110018
Top 5 accuracy 0.9962738168408113
UER  0.032536877688998156


 Completed: 0.00%
Top 1 accuracy 0.9665027658266748
Top 5 accuracy 0.9963122311001844
UER  0.033497234173325174


 Completed: 0.00%
Top 1 accuracy 0.9644283958205285
Top 5 accuracy 0.9959280885064536
UER  0.03557160417947147


 Completed: 0.00%
Top 1 accuracy 0.9622771972956361
Top 5 accuracy 0.9958512599877074
UER  0.03

Top 1 accuracy 0.37488475722188075
Top 5 accuracy 0.8286724031960664
UER  0.6251152427781193


 Completed: 0.00%
Top 1 accuracy 0.3707744314689613
Top 5 accuracy 0.8270974185617701
UER  0.6292255685310387


 Completed: 0.00%
Top 1 accuracy 0.36578057775046097
Top 5 accuracy 0.8238706207744315
UER  0.634219422249539


 Completed: 0.00%
Top 1 accuracy 0.35828979717271053
Top 5 accuracy 0.8192224953902889
UER  0.6417102028272894


 Completed: 0.00%
Top 1 accuracy 0.3505301167793485
Top 5 accuracy 0.8134219422249539
UER  0.6494698832206516


 Completed: 0.00%
Top 1 accuracy 0.3409265519360787
Top 5 accuracy 0.8078134603564844
UER  0.6590734480639213


Loading svhn/resnet18_P3-0.3.pth
 Completed: 0.00%
Top 1 accuracy 0.9359634296250768
Top 5 accuracy 0.9927781192378611
UER  0.06403657037492316


 Completed: 0.00%
Top 1 accuracy 0.9363475722188076
Top 5 accuracy 0.9927781192378611
UER  0.0636524277811924


 Completed: 0.00%
Top 1 accuracy 0.9366164720344192
Top 5 accuracy 0.9928933620159803


 Completed: 0.00%
Top 1 accuracy 0.87450061462815
Top 5 accuracy 0.9914336201598033
UER  0.12549938537185001


 Completed: 0.00%
Top 1 accuracy 0.8736555009219422
Top 5 accuracy 0.991279963122311
UER  0.12634449907805778


 Completed: 0.00%
Top 1 accuracy 0.8720036877688998
Top 5 accuracy 0.9911263060848187
UER  0.1279963122311002


 Completed: 0.00%
Top 1 accuracy 0.8690073755377996
Top 5 accuracy 0.9909342347879533
UER  0.1309926244622004


 Completed: 0.00%
Top 1 accuracy 0.8647433927473879
Top 5 accuracy 0.9905500921942225
UER  0.13525660725261213


 Completed: 0.00%
Top 1 accuracy 0.8579824830977258
Top 5 accuracy 0.9897049784880148
UER  0.14201751690227415




## View Evaluation Results

In [6]:
df = pd.read_csv(dir_results + 'eval_noise.csv', index_col = 0)

In [13]:
df[(df['dataset'] == 'cifar10') & (df['atk_name'] == 'chess') & (df['eps'] == 8)].sort_values(by = 'UER')

,model_name,dataset,atk_name,atk_param,eps,top1acc,top5acc,UER,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
15,resnet18_sfP,cifar10,chess,-1,8,0.7885,0.9882,0.2115,0.0902,0.0964,0.1033,0.1527,0.0786,0.1005,0.0940,0.0826,0.1037,0.0980
39,resnet18_sfP-cutout,cifar10,chess,-1,8,0.6984,0.9778,0.3016,0.0644,0.1017,0.2041,0.1094,0.0604,0.0478,0.1612,0.0767,0.0735,0.1008
27,resnet18_sfP-mixup,cifar10,chess,-1,8,0.6098,0.9407,0.3902,0.0994,0.0963,0.1111,0.0809,0.0401,0.0361,0.3469,0.0455,0.0920,0.0517
51,resnet18_P2-0.3,cifar10,chess,-1,8,0.2396,0.6641,0.7604,0.4082,0.0377,0.0328,0.2691,0.0021,0.0002,0.0120,0.0017,0.0602,0.1760
99,resnet18_P3-0.6,cifar10,chess,-1,8,0.2387,0.6785,0.7613,0.6903,0.0569,0.0126,0.0208,0.0020,0.0006,0.0176,0.0011,0.0250,0.1731
3,resnet18,cifar10,chess,-1,8,0.2326,0.6466,0.7674,0.3978,0.0509,0.0277,0.0945,0.0018,0.0008,0.0105,0.0020,0.0348,0.3792
111,resnet18_P3-0.9,cifar10,chess,-1,8,0.2323,0.6637,0.7677,0.3179,0.0416,0.0045,0.5905,0.0006,0.0000,0.0251,0.0000,0.0029,0.0169
123,resnet18_P4-0.3,cifar10,chess,-1,8,0.2314,0.6338,0.7686,0.3661,0.0514,0.0255,0.0979,0.0019,0.0009,0.0097,0.0016,0.0390,0.4060
135,resnet18_P4-0.6,cifar10,chess,-1,8,0.2275,0.6209,0.7725,0.3541,0.0515,0.0261,0.0851,0.0023,0.0009,0.0100,0.0014,0.0341,0.4345
87,resnet18_P3-0.3,cifar10,chess,-1,8,0.2260,0.6572,0.7740,0.5247,0.0513,0.0206,0.0367,0.0024,0.0008,0.0119,0.0020,0.0319,0.3177


In [14]:
df[(df['dataset'] == 'cifar10') & (df['atk_name'] == 'random') & (df['eps'] == 8)].sort_values(by = 'UER')

,model_name,dataset,atk_name,atk_param,eps,top1acc,top5acc,UER,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
9,resnet18,cifar10,random,-1,8,0.8429,0.9906,0.1571,0.0717,0.0935,0.1286,0.0973,0.1104,0.0718,0.1396,0.0861,0.1039,0.0971
129,resnet18_P4-0.3,cifar10,random,-1,8,0.8421,0.9899,0.1579,0.0696,0.0933,0.1273,0.0996,0.1140,0.0732,0.1359,0.0837,0.1054,0.0980
93,resnet18_P3-0.3,cifar10,random,-1,8,0.8336,0.9882,0.1664,0.0744,0.0929,0.1205,0.0900,0.1144,0.0657,0.1521,0.0839,0.1072,0.0989
141,resnet18_P4-0.6,cifar10,random,-1,8,0.8317,0.9892,0.1683,0.0674,0.0920,0.1291,0.0947,0.1208,0.0730,0.1419,0.0795,0.1017,0.0999
33,resnet18_sfP-mixup,cifar10,random,-1,8,0.7985,0.9735,0.2015,0.0884,0.0979,0.1013,0.0687,0.1024,0.0750,0.1804,0.0852,0.1014,0.0993
153,resnet18_P4-0.9,cifar10,random,-1,8,0.7912,0.9879,0.2088,0.0673,0.0860,0.1185,0.0700,0.1188,0.0717,0.1972,0.0729,0.0955,0.1021
45,resnet18_sfP-cutout,cifar10,random,-1,8,0.7857,0.9873,0.2143,0.0865,0.0944,0.1443,0.1054,0.0835,0.0685,0.1185,0.0885,0.0997,0.1107
105,resnet18_P3-0.6,cifar10,random,-1,8,0.7849,0.9830,0.2151,0.0808,0.0918,0.1043,0.0734,0.1106,0.0587,0.2108,0.0702,0.1077,0.0917
21,resnet18_sfP,cifar10,random,-1,8,0.7667,0.9853,0.2333,0.0867,0.0905,0.0990,0.1456,0.0861,0.0992,0.1039,0.0754,0.1137,0.0999
57,resnet18_P2-0.3,cifar10,random,-1,8,0.7607,0.9752,0.2393,0.0596,0.0858,0.1367,0.0872,0.1303,0.0507,0.1698,0.0689,0.1271,0.0839


In [12]:
df[(df['dataset'] == 'svhn') & (df['atk_name'] == 'chess') & (df['eps'] == 8)].sort_values(by = 'UER')

,model_name,dataset,atk_name,atk_param,eps,top1acc,top5acc,UER,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
159,resnet18,svhn,chess,-1,8,0.942340,0.993662,0.057660,0.067609,0.198486,0.160341,0.108904,0.095498,0.090773,0.075215,0.077289,0.062193,0.063691
171,resnet18_sfP,svhn,chess,-1,8,0.968231,0.996274,0.031769,0.069568,0.197987,0.156154,0.113322,0.096612,0.091810,0.076829,0.075522,0.061809,0.060387
183,resnet18_sfP-mixup,svhn,chess,-1,8,0.968692,0.995851,0.031308,0.068838,0.195682,0.159073,0.109481,0.096727,0.091580,0.078096,0.078788,0.061540,0.060195
195,resnet18_sfP-cutout,svhn,chess,-1,8,0.970882,0.996696,0.029118,0.067686,0.194491,0.159919,0.114743,0.096381,0.089006,0.076406,0.077597,0.062884,0.060887
207,resnet18_P2-0.3,svhn,chess,-1,8,0.937001,0.993431,0.062999,0.069415,0.194837,0.158344,0.111901,0.095037,0.091157,0.075830,0.077981,0.062308,0.063191
219,resnet18_P2-0.6,svhn,chess,-1,8,0.895667,0.988745,0.104333,0.088430,0.188998,0.142248,0.118969,0.086893,0.091195,0.075945,0.078634,0.065074,0.063614
231,resnet18_P2-0.9,svhn,chess,-1,8,0.379955,0.829402,0.620045,0.462277,0.119584,0.019015,0.058467,0.009335,0.036993,0.059657,0.022780,0.193646,0.018247
243,resnet18_P3-0.3,svhn,chess,-1,8,0.936809,0.993009,0.063191,0.068416,0.193685,0.161455,0.107598,0.094192,0.092617,0.077750,0.078749,0.063038,0.062500
255,resnet18_P3-0.6,svhn,chess,-1,8,0.915642,0.989436,0.084358,0.062308,0.192379,0.157345,0.113053,0.087316,0.090619,0.086125,0.082130,0.071835,0.056892
267,resnet18_P3-0.9,svhn,chess,-1,8,0.496927,0.883490,0.503073,0.012216,0.110748,0.087585,0.417678,0.011716,0.060310,0.063576,0.011601,0.211701,0.012869


In [11]:
df[(df['dataset'] == 'svhn') & (df['atk_name'] == 'random') & (df['eps'] == 8)].sort_values(by = 'UER')

,model_name,dataset,atk_name,atk_param,eps,top1acc,top5acc,UER,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
165,resnet18,svhn,random,-1,8,0.936002,0.993354,0.063998,0.066994,0.198256,0.159342,0.108712,0.095152,0.091848,0.076944,0.076598,0.062039,0.064113
177,resnet18_sfP,svhn,random,-1,8,0.964428,0.995928,0.035572,0.068954,0.198794,0.155616,0.112631,0.096458,0.092386,0.077597,0.075522,0.061501,0.060541
189,resnet18_sfP-mixup,svhn,random,-1,8,0.949408,0.994199,0.050592,0.068339,0.198256,0.162339,0.111171,0.096996,0.090197,0.083013,0.075061,0.059004,0.055624
201,resnet18_sfP-cutout,svhn,random,-1,8,0.961663,0.995736,0.038337,0.065842,0.198640,0.158305,0.117509,0.096842,0.088007,0.076829,0.077213,0.060695,0.060118
213,resnet18_P2-0.3,svhn,random,-1,8,0.928626,0.992663,0.071374,0.068915,0.193838,0.156269,0.113168,0.095191,0.092386,0.077097,0.077059,0.062730,0.063345
225,resnet18_P2-0.6,svhn,random,-1,8,0.879994,0.985748,0.120006,0.090773,0.186194,0.137408,0.122234,0.085549,0.094153,0.076329,0.076521,0.067340,0.063499
237,resnet18_P2-0.9,svhn,random,-1,8,0.358290,0.819222,0.641710,0.476875,0.107483,0.016211,0.053127,0.008490,0.034919,0.061655,0.018593,0.205478,0.017171
249,resnet18_P3-0.3,svhn,random,-1,8,0.930240,0.992817,0.069760,0.067686,0.193262,0.160380,0.107061,0.094153,0.094153,0.079325,0.077904,0.062769,0.063307
261,resnet18_P3-0.6,svhn,random,-1,8,0.906845,0.988514,0.093155,0.061040,0.190343,0.157037,0.113360,0.085741,0.092309,0.090504,0.080977,0.071988,0.056699
273,resnet18_P3-0.9,svhn,random,-1,8,0.483482,0.874424,0.516518,0.011255,0.108712,0.080439,0.430893,0.010794,0.060349,0.064613,0.010679,0.209819,0.012446
